# Import libraries

In [29]:
import pandas as pd
import seaborn as sns
import session_info
from sklearn import compose
from sklearn.linear_model import Ridge
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [30]:
try:
    session_info_output = session_info.show()
except Exception as error:
    print("The session info has been requested already:")
    
session_info_output

The session info has been requested already:


/home/ubuntu/anaconda3/envs/research-on-argentina/lib/python3.11/site-packages/session_info/main.py:213: DeprecationWarning: Accessing jsonschema.__version__ is deprecated and will be removed in a future release. Use importlib.metadata directly to query for jsonschema's version.
  mod_version = _find_version(mod.__version__)


In [31]:
import janitor # This library generates an error when I execute session_info()

# Import external files

- Extension of the "missing" functions from Pandas
- DataFrame and dictionary functions
- Statistical functions

In [32]:
%run utils/u.0.0-pandas_missing_extension.ipynb
%run utils/u.0.1-df_functions.ipynb
%run utils/u.0.2-statistical-functions.ipynb

# Get the processed data (V2)

In [33]:
arg_di_df = pd.read_csv('../data/processed/WDICSV_PROCESSED_V2.csv').clean_names(case_type="snake")

print(arg_di_df.shape)
arg_di_df.info()

(54, 28)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    54 non-null     int64  
 1   it_mlt_main             54 non-null     float64
 2   ny_gdp_mktp_kd_zg       54 non-null     float64
 3   ny_gdp_pcap_kd          54 non-null     float64
 4   ny_gdp_pcap_kd_zg       54 non-null     float64
 5   sp_dyn_le00_in          53 non-null     float64
 6   it_cel_sets             54 non-null     float64
 7   sh_dth_imrt             53 non-null     float64
 8   sp_pop_0014_to_zs       54 non-null     float64
 9   sp_pop_1564_to_zs       54 non-null     float64
 10  sp_pop_65_up_to_zs      54 non-null     float64
 11  sp_pop_totl             54 non-null     float64
 12  sm_pop_refg             54 non-null     float64
 13  sm_pop_refg_or          44 non-null     float64
 14  se_pre_enrr             45 non-null

# Get columns names and types

In [34]:
indicators_names, categorical_cols, numeric_cols, columns_with_missing_values = get_columns(arg_di_df)

In [35]:
print(f'Total Columns: {len(indicators_names)}')
print(f'Categorical Columns: {len(categorical_cols)}')
print(f'Numeric Columns: {len(numeric_cols)}')

numeric_cols = list(numeric_cols)

Total Columns: 28
Categorical Columns: 6
Numeric Columns: 22


In [36]:
indicators_names

['year, Year',
 'it_mlt_main, Fixed telephone subscriptions',
 'ny_gdp_mktp_kd_zg, GDP growth (annual %)',
 'ny_gdp_pcap_kd, GDP per capita (constant 2015 US$)',
 'ny_gdp_pcap_kd_zg, GDP per capita growth (annual %)',
 'sp_dyn_le00_in, Life expectancy at birth, total (years)',
 'it_cel_sets, Mobile cellular subscriptions',
 'sh_dth_imrt, Number of infant deaths',
 'sp_pop_0014_to_zs, Population ages 0-14 (% of total population)',
 'sp_pop_1564_to_zs, Population ages 15-64 (% of total population)',
 'sp_pop_65_up_to_zs, Population ages 65 and above (% of total population)',
 'sp_pop_totl, Population, total',
 'sm_pop_refg, Refugee population by country or territory of asylum',
 'sm_pop_refg_or, Refugee population by country or territory of origin',
 'se_pre_enrr, School enrollment, preprimary (% gross)',
 'se_prm_enrr, School enrollment, primary (% gross)',
 'se_sec_enrr, School enrollment, secondary (% gross)',
 'se_ter_enrr, School enrollment, tertiary (% gross)',
 'sp_urb_totl_in_zs,

In [37]:
columns_with_missing_values

['sm_pop_refg_or',
 'se_ter_enrr',
 'se_pre_enrr',
 'se_prm_enrr',
 'se_sec_enrr',
 'sp_dyn_le00_in',
 'sh_dth_imrt']

# Target variable:

sp_dyn_le00_in (Life expectancy at birth, total (years))

In [38]:
target_variable = "sp_dyn_le00_in"
target_variable_name = get_indicator_name(target_variable, True)

# Process data

## Ordinal coding

This method implies to transform every category into an integer. It's useful when the categories have an order.

### Apply to a group of columns

Column transformer: Applies transformers to columns of an array or pandas DataFrame.

make_column_transformer(): Construct a ColumnTransformer from the given transformers. This is a shorthand for the ColumnTransformer constructor; it does not require, and does not permit, naming the transformers. Instead, they will be given names automatically based on their types.


In [39]:
categorical_variables_ordered_values = [list(item.keys()) for item in df_categorical_values["Indicator Colors"]]

for item in categorical_variables_ordered_values:
    item.remove("Unknown")
    item.insert(0, "Unknown")

In [40]:
ordinal_encoder = OrdinalEncoder(categories= categorical_variables_ordered_values)

In [41]:
categorical_transformer = compose.make_column_transformer(
    # Tuples of the form (transformer, columns) specifying the transformer objects to be applied to subsets of the data
    (ordinal_encoder, df_categorical_values["Indicator Name"]),
    remainder="passthrough"
    # By specifying remainder='passthrough', all remaining columns that were not specified in transformers,
    # but present in the data passed to fit will be automatically passed through. This subset of columns is
    # concatenated with the output of the transformers. For dataframes, extra columns not seen during fit
    # will be excluded from the output of transform. 
)

In [42]:
arg_di_df_transformed = (
    pd.DataFrame(
        categorical_transformer.fit_transform(arg_di_df),
        columns=categorical_transformer.get_feature_names_out(),
        index=arg_di_df.index
    )
    .rename(
        columns=remove_prefix
    )
)

In [43]:
categorical_transformer

/home/ubuntu/anaconda3/envs/research-on-argentina/lib/python3.11/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('ordinalencoder',
                                 OrdinalEncoder(categories=[['Unknown',
                                                             'Negative growth',
                                                             'Low growth',
                                                             'Moderate growth',
                                                             'High growth'],
                                                            ['Unknown',
                                                             'Low life '
                                                             'expectancy',
                                                             'Medium life '
                                                             'expectancy',
                                                             'High life '
                                                             'expectancy'],
                                                            ['Unknown',
                                                             'Low income',
                                                             'Middle income',
                                                             'High income'],
                                                            ['Unknown', 'Low',
                                                             'Medium', 'High'],
                                                            ['Unknown',
                                                             'High volatility',
                                                             'Medium '
                                                             'volatility',
                                                             'Low volatility'],
                                                            ['Unknown', 'Low',
                                                             'Medium', 'High',
                                                             'Very High']]),
                                 0     ny_gdp_mktp_kd_zg_cat
1        sp_dyn_le00_in_cat
2        ny_gdp_pcap_kd_cat
3           se_sec_enrr_cat
4    ny_gdp_mktp_kd_grw_cat
5           it_cel_sets_cat
Name: Indicator Name, dtype: object)])

In [44]:
encoder = categorical_transformer.named_transformers_['ordinalencoder']
# Check categories for each column
for col_name, categories in zip(list(df_categorical_values["Indicator Name"]), encoder.categories_):
    print(f"{col_name}:")
    for i, cat in enumerate(categories):
        print(f"  {cat} → {i}")

ny_gdp_mktp_kd_zg_cat:
  Unknown → 0
  Negative growth → 1
  Low growth → 2
  Moderate growth → 3
  High growth → 4
sp_dyn_le00_in_cat:
  Unknown → 0
  Low life expectancy → 1
  Medium life expectancy → 2
  High life expectancy → 3
ny_gdp_pcap_kd_cat:
  Unknown → 0
  Low income → 1
  Middle income → 2
  High income → 3
se_sec_enrr_cat:
  Unknown → 0
  Low → 1
  Medium → 2
  High → 3
ny_gdp_mktp_kd_grw_cat:
  Unknown → 0
  High volatility → 1
  Medium volatility → 2
  Low volatility → 3
it_cel_sets_cat:
  Unknown → 0
  Low → 1
  Medium → 2
  High → 3
  Very High → 4


In [45]:
arg_di_df_transformed[categorical_cols].head(5)

,ny_gdp_mktp_kd_zg_cat,sp_dyn_le00_in_cat,it_cel_sets_cat,ny_gdp_pcap_kd_cat,se_sec_enrr_cat,ny_gdp_mktp_kd_grw_cat
0,3.0,2.0,1.0,2.0,2.0,0.0
1,4.0,2.0,0.0,2.0,2.0,0.0
2,2.0,2.0,0.0,2.0,2.0,2.0
3,3.0,2.0,0.0,2.0,2.0,3.0
4,4.0,2.0,0.0,2.0,2.0,3.0


In [46]:
arg_di_df_transformed[categorical_cols].describe()

,ny_gdp_mktp_kd_zg_cat,sp_dyn_le00_in_cat,it_cel_sets_cat,ny_gdp_pcap_kd_cat,se_sec_enrr_cat,ny_gdp_mktp_kd_grw_cat
count,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,2.481481,2.648148,1.944444,2.296296,2.407407,1.277778
std,1.342214,0.587846,1.471960,0.460911,0.961887,0.626962
min,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000
50%,3.000000,3.000000,1.000000,2.000000,3.000000,1.000000
75%,4.000000,3.000000,4.000000,3.000000,3.000000,1.750000
max,4.000000,3.000000,4.000000,3.000000,3.000000,3.000000


In [47]:
print("There is no categorical variables in the transformed data")
arg_di_df_transformed.select_dtypes(include=['object']).columns

There is no categorical variables in the transformed data


Index([], dtype='object')

## Scale Numerical Features

In [48]:
if 'year' in numeric_cols:
    numeric_cols.remove('year')

In [49]:
scaler = StandardScaler()
arg_di_df_transformed[numeric_cols] = scaler.fit_transform(arg_di_df_transformed[numeric_cols])

In [50]:
arg_di_df_transformed[numeric_cols].describe()

,it_mlt_main,ny_gdp_mktp_kd_zg,ny_gdp_pcap_kd,ny_gdp_pcap_kd_zg,sp_dyn_le00_in,it_cel_sets,sh_dth_imrt,sp_pop_0014_to_zs,sp_pop_1564_to_zs,sp_pop_65_up_to_zs,...,sm_pop_refg,sm_pop_refg_or,se_pre_enrr,se_prm_enrr,se_sec_enrr,se_ter_enrr,sp_urb_totl_in_zs,year_of_dictatorship,it_cel_sets_pct,dem_dep_cat
count,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.300000e+01,5.400000e+01,5.300000e+01,5.400000e+01,5.400000e+01,5.400000e+01,...,5.400000e+01,4.400000e+01,4.500000e+01,4.700000e+01,4.800000e+01,4.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01,5.400000e+01
mean,6.476301e-17,-5.756712e-17,-5.345518e-16,2.467162e-17,2.853064e-15,-2.878356e-17,-1.717704e-16,8.552829e-16,3.569161e-15,1.727014e-16,...,5.139921e-17,-4.163336e-17,1.973730e-16,1.758641e-15,-3.376928e-16,-1.917658e-16,-1.110223e-16,6.167906e-17,7.401487e-17,-1.069104e-16
std,1.009390e+00,1.009390e+00,1.009390e+00,1.009390e+00,1.009570e+00,1.009390e+00,1.009570e+00,1.009390e+00,1.009390e+00,1.009390e+00,...,1.009390e+00,1.011561e+00,1.011300e+00,1.010811e+00,1.010582e+00,1.011561e+00,1.009390e+00,1.009390e+00,1.009390e+00,1.009390e+00
min,-1.346503e+00,-2.381731e+00,-1.609576e+00,-2.332194e+00,-1.924073e+00,-7.638043e-01,-1.295161e+00,-2.067495e+00,-1.406246e+00,-1.770959e+00,...,-8.275976e-01,-1.005516e+00,-2.068652e+00,-1.369023e+00,-1.656433e+00,-1.442387e+00,-2.114291e+00,-5.345225e-01,-7.747653e-01,-1.852406e+00
25%,-9.994729e-01,-7.555036e-01,-7.684132e-01,-7.776206e-01,-8.991756e-01,-7.638043e-01,-7.975029e-01,-9.073865e-01,-9.643474e-01,-8.476192e-01,...,-7.487734e-01,-8.050253e-01,-7.112004e-01,-1.063677e+00,-1.036507e+00,-1.021893e+00,-7.660271e-01,-5.345225e-01,-7.747653e-01,-8.710780e-01
50%,2.626002e-01,1.253846e-01,-3.636115e-01,8.627998e-02,2.163184e-01,-7.133432e-01,-1.985796e-01,3.552099e-01,7.998749e-03,1.219308e-01,...,-1.884167e-01,-4.760948e-01,1.388185e-01,1.853656e-02,-3.954943e-02,-1.211492e-02,2.697168e-01,-5.345225e-01,-7.136044e-01,-2.923399e-02
75%,1.097088e+00,7.512209e-01,9.845795e-01,7.374341e-01,9.088391e-01,1.281157e+00,4.151685e-01,8.451986e-01,8.644281e-01,7.550340e-01,...,8.914206e-02,7.229350e-01,8.429890e-01,8.755528e-01,7.982536e-01,8.403892e-01,8.528474e-01,-5.345225e-01,1.254396e+00,9.644444e-01
max,1.370511e+00,1.598804e+00,1.932067e+00,1.684094e+00,1.369678e+00,1.778019e+00,1.939047e+00,1.165843e+00,1.903152e+00,1.763496e+00,...,2.697312e+00,2.521871e+00,1.477965e+00,1.596394e+00,1.631297e+00,2.030766e+00,1.264111e+00,1.870829e+00,1.831949e+00,1.430109e+00


# Impute missing values

MICE (Multiple Imputation by Chained Equations) is a technique for imputing missing data, where multiple possible values are generated for each missing entry using iterative regression models.

If dependent variables have NA values, then linear regression will predict NA results

In [51]:
def impute_with_mice(data, target_variable, numeric_cols):    
    rows_with_na_target_variable = data[data[target_variable].isna()].copy()
    rows_with_na_target_variable = list(rows_with_na_target_variable.index)
    if target_variable in numeric_cols:
        numeric_cols.remove(target_variable)
        
    mice_imputer = IterativeImputer(estimator=Ridge())

    # data[numeric_cols] = pd.DataFrame(mice_imputer.fit_transform(data[numeric_cols]), columns=numeric_cols)
    return pd.DataFrame(mice_imputer.fit_transform(data[numeric_cols]), columns=numeric_cols)

In [52]:
arg_di_df_transformed[numeric_cols] = impute_with_mice(arg_di_df_transformed, target_variable, numeric_cols)

In [53]:
print("Check missing values in all columns")
print("The unique Nan value is the target variable in one row")
arg_di_df_transformed[arg_di_df_transformed.isna().any(axis=1)]

Check missing values in all columns
The unique Nan value is the target variable in one row


,ny_gdp_mktp_kd_zg_cat,sp_dyn_le00_in_cat,ny_gdp_pcap_kd_cat,se_sec_enrr_cat,ny_gdp_mktp_kd_grw_cat,it_cel_sets_cat,year,it_mlt_main,ny_gdp_mktp_kd_zg,ny_gdp_pcap_kd,...,sm_pop_refg,sm_pop_refg_or,se_pre_enrr,se_prm_enrr,se_sec_enrr,se_ter_enrr,sp_urb_totl_in_zs,year_of_dictatorship,it_cel_sets_pct,dem_dep_cat
53,1.0,0.0,3.0,0.0,1.0,4.0,2023.0,0.412407,-0.66078,1.011066,...,-0.68224,-0.184756,1.424528,-0.03874,1.578337,1.890704,1.264111,-0.534522,1.418794,-1.852406


In [26]:
arg_di_df.iloc[rows_with_na_target_variable][target_variable]

53   NaN
Name: sp_dyn_le00_in, dtype: float64

# Save the processed data (V3)

In [27]:
arg_di_df_transformed.to_csv('../data/processed/WDICSV_PROCESSED_V3.csv', index=False)